In [5]:
import pandas as pd

df = pd.read_csv('./data/mortgage-300h-360m-5r.csv')
print(df.shape)
print(df.columns.tolist())

(361, 5)
['Period', 'Monthly Payment', 'Computed Interest Due', 'Principal Due', 'Principal Balance']


In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

# LLM from Ollama
local_model = "mistral:latest"
llm = ChatOllama(model=local_model, temperature=0)
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['Period'].mean()")

np.float64(180.0)

In [7]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
response = llm_with_tools.invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Monthly Payment' and 'Principal Balance' columns"
)
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral:latest', 'created_at': '2024-12-26T17:46:59.3881021Z', 'done': True, 'done_reason': 'stop', 'total_duration': 38619157500, 'load_duration': 25513200, 'prompt_eval_count': 141, 'prompt_eval_duration': 2578000000, 'eval_count': 181, 'eval_duration': 36014000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-8853e663-75c8-486a-a20c-e90fd9b89517-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd\n\ndf = pd.DataFrame(\n {\n \t'Monthly Payment': [100, 200, 300, 400, 500],\n \t'Principal Balance': [5000, 4000, 3000, 2000, 1000]\n }\n)\ndf.corr()['Monthly Payment']['Principal Balance']"}, 'id': '2fd93843-42da-41f4-b4fb-f79ebf50c434', 'type': 'tool_call'}], usage_metadata={'input_tokens': 141, 'output_tokens': 181, 'total_tokens': 322})

In [8]:
response.tool_calls

[{'name': 'python_repl_ast',
  'args': {'query': "import pandas as pd\n\ndf = pd.DataFrame(\n {\n \t'Monthly Payment': [100, 200, 300, 400, 500],\n \t'Principal Balance': [5000, 4000, 3000, 2000, 1000]\n }\n)\ndf.corr()['Monthly Payment']['Principal Balance']"},
  'id': '2fd93843-42da-41f4-b4fb-f79ebf50c434',
  'type': 'tool_call'}]

In [11]:
from langchain_core.output_parsers import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
(llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Monthly Payment' and 'Principal Balance' columns"
)

{'query': "import pandas as pd\n\ndf = pd.DataFrame(\n {\n \t'Monthly Payment': [100, 200, 300, 400, 500],\n \t'Principal Balance': [5000, 4000, 3000, 2000, 1000]\n }\n)\ndf.corr()['Monthly Payment']['Principal Balance']"}

In [13]:
system = f"""You have access to a pandas dataframe `df`. 
Here is the output of `df.head().to_markdown()`:

{df.head().to_markdown()}

Given a user question, write the Python code to answer it. 
Return ONLY the valid Python code and nothing else. 
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llm_with_tools
response = code_chain.invoke({"question": "What's is the value of Principal Balance at the last period?"})
print(response)

content=" The following Python code will return the value of Principal Balance at the last period:\n\n```python\nlast_period_principal_balance = df.loc[df.shape[0] - 1, 'Principal Balance']\nprint(last_period_principal_balance)\n```" additional_kwargs={} response_metadata={'model': 'mistral:latest', 'created_at': '2024-12-26T17:52:19.9341727Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11761023100, 'load_duration': 18722300, 'prompt_eval_count': 468, 'prompt_eval_duration': 707000000, 'eval_count': 69, 'eval_duration': 11032000000, 'message': Message(role='assistant', content=" The following Python code will return the value of Principal Balance at the last period:\n\n```python\nlast_period_principal_balance = df.loc[df.shape[0] - 1, 'Principal Balance']\nprint(last_period_principal_balance)\n```", images=None, tool_calls=None)} id='run-e821c12d-98ad-4470-b20f-cfca82a564d6-0' usage_metadata={'input_tokens': 468, 'output_tokens': 69, 'total_tokens': 537}


In [28]:
chain = prompt | llm_with_tools | tool
print(chain.invoke({"question": "What's the correlation between Monthly Payment and Principal Balance"}))

ValidationError: 1 validation error for PythonInputs
  Input should be a valid dictionary or instance of PythonInputs [type=model_type, input_value=AIMessage(content=' Here ...3, 'total_tokens': 539}), input_type=AIMessage]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [37]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe `df`.
Here is the output of `df.head().to_markdown()`:

{df.head().to_markdown()}

Given a user question, write the Python code to answer it. 
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser | tool)
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["tool_output", "response"])
)

In [38]:
print(chain.invoke({"question": "What's the correlation between Monthly Payment and Principal Balance"}))

ValidationError: 1 validation error for PythonInputs
  Input should be a valid dictionary or instance of PythonInputs [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [1]:
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

# LLM from Ollama
local_model = "mistral:latest"
llm = ChatOllama(model=local_model, temperature=0)

df = pd.read_csv('./data/mortgage-300h-360m-5r.csv')
print(df.shape)
print(df.columns.tolist())
pd_agent = create_pandas_dataframe_agent(llm, df, agent_type="tool-calling", verbose=True, allow_dangerous_code=True)
# pd_agent.run("what is the mean of the profit?")
query = "What's the correlation between Monthly Payment and Principal Balance?"
query = query + " using tool python_repl_ast"
pd_agent.invoke(query)

(361, 5)
['Period', 'Monthly Payment', 'Computed Interest Due', 'Principal Due', 'Principal Balance']


> Entering new AgentExecutor chain...


C:\Users\JTOSH\AppData\Local\Temp\ipykernel_17140\1877157237.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pd_agent.run(query)



Invoking: `python_repl_ast` with `{'query': "correlation = df['Monthly Payment'].corr(df['Principal Balance']) ; print(correlation)"}`


-0.0704212874193625
 The correlation between Monthly Payment and Principal Balance is approximately -0.07, which suggests a weak negative correlation. This means that as the Principal Balance increases, there is a slight decrease in the Monthly Payment, but the relationship is not strong enough to be considered significant.

> Finished chain.


' The correlation between Monthly Payment and Principal Balance is approximately -0.07, which suggests a weak negative correlation. This means that as the Principal Balance increases, there is a slight decrease in the Monthly Payment, but the relationship is not strong enough to be considered significant.'

In [2]:
query = "What's the Principal Balance on 360 period?"
query = query + " using tool python_repl_ast"
pd_agent.invoke(query)



> Entering new AgentExecutor chain...
 To find the Principal Balance on the 360th period, we can use the `iloc` function to access the row corresponding to the 360th index. Here is the code snippet:

```python
principal_balance = df.iloc[359]['Principal Balance']
print(principal_balance)
```

This will give you the Principal Balance on the 360th period.

> Finished chain.


{'input': "What's the Principal Balance on 360 period? using tool python_repl_ast",
 'output': " To find the Principal Balance on the 360th period, we can use the `iloc` function to access the row corresponding to the 360th index. Here is the code snippet:\n\n```python\nprincipal_balance = df.iloc[359]['Principal Balance']\nprint(principal_balance)\n```\n\nThis will give you the Principal Balance on the 360th period."}

In [3]:
query = "What's is the value of Principal Balance on 360 period?"
query = query + " using tool python_repl_ast"
pd_agent.invoke(query)



> Entering new AgentExecutor chain...
 To find the value of Principal Balance on the 360th period, we can use the `.loc` function to select the row with index 360 and access the 'Principal Balance' column:

```python
principal_balance_at_360 = df.loc[360, "Principal Balance"]
```

Since we don't know the exact index value for the 360th period, let's assume that the Period is incremented by one every month and find the index of the row corresponding to the 360th month:

```python
import datetime as dt

start_date = df.iloc[0]['Period']
end_date = df.iloc[-1]['Period'] + 1
periods_count = (end_date - start_date) / dt.timedelta(days=30)
index_at_360 = int((360 - start_date) / dt.timedelta(days=30))
```

Now, we can use the index to find the value of Principal Balance at the 360th period:

```python
principal_balance_at_360 = df.loc[index_at_360, "Principal Balance"]
```

> Finished chain.


{'input': "What's is the value of Principal Balance on 360 period? using tool python_repl_ast",
 'output': ' To find the value of Principal Balance on the 360th period, we can use the `.loc` function to select the row with index 360 and access the \'Principal Balance\' column:\n\n```python\nprincipal_balance_at_360 = df.loc[360, "Principal Balance"]\n```\n\nSince we don\'t know the exact index value for the 360th period, let\'s assume that the Period is incremented by one every month and find the index of the row corresponding to the 360th month:\n\n```python\nimport datetime as dt\n\nstart_date = df.iloc[0][\'Period\']\nend_date = df.iloc[-1][\'Period\'] + 1\nperiods_count = (end_date - start_date) / dt.timedelta(days=30)\nindex_at_360 = int((360 - start_date) / dt.timedelta(days=30))\n```\n\nNow, we can use the index to find the value of Principal Balance at the 360th period:\n\n```python\nprincipal_balance_at_360 = df.loc[index_at_360, "Principal Balance"]\n```'}